# Push Down Queries

Sample that shows how to push queries to Azure SQL

Define variables used thoughout the script. Azure Key Value has been used to securely store sensitive data. More info here: [Create an Azure Key Vault-backed secret scope](https://docs.microsoft.com/en-us/azure/databricks/security/secrets/secret-scopes#--create-an-azure-key-vault-backed-secret-scope)

In [3]:
val scope = "key-vault-secrets"

val server = dbutils.secrets.get(scope, "srv001")
val database = "ApacheSpark"

val jdbcUrl = s"jdbc:sqlserver://$server.database.windows.net;database=$database;"

val connectionProperties = new java.util.Properties()
connectionProperties.put("user", dbutils.secrets.get(scope, "dbuser001"))
connectionProperties.put("password", dbutils.secrets.get(scope, "dbpwd001"))
connectionProperties.setProperty("Driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")

scope: String = key-vault-secrets
server: String = [REDACTED]
database: String = ApacheSpark
jdbcUrl: String = jdbc:sqlserver://[REDACTED].database.windows.net;database=ApacheSpark;
connectionProperties: java.util.Properties = {user=[REDACTED], password=[REDACTED], Driver=com.microsoft.sqlserver.jdbc.SQLServerDriver}
res0: Object = null

A pushdown query is executed as a subquery by Azure SQL, so you *MUST* alias the subquery and put it in parenthesis: `(<...>) AS subquery`

In [5]:
val pushDown = """(
SELECT
	L_COMMITDATE,
	COUNT(*) AS TotalOrders
FROM
	dbo.LINEITEM
GROUP BY
	L_COMMITDATE
) AS SubQuery"""

val li = spark.read.jdbc(jdbcUrl, pushDown, connectionProperties)

pushDown: String =
(
SELECT
	L_COMMITDATE,
	COUNT(*) AS TotalOrders
FROM
	dbo.LINEITEM
GROUP BY
	L_COMMITDATE
) AS SubQuery
li: org.apache.spark.sql.DataFrame = [L_COMMITDATE: date, TotalOrders: int]

In [6]:
li.printSchema

root
-- L_COMMITDATE: date (nullable = true)
-- TotalOrders: integer (nullable = true)

In [7]:
display(li.where("L_COMMITDATE='1995-01-01'"))

L_COMMITDATE,TotalOrders
1995-01-01,24781
